## Defining a function to download a Youtube video (in audio format) by name search query

In [7]:
from youtube_search import YoutubeSearch

def get_video_link_by_name(video_name):
    results = YoutubeSearch(video_name, max_results=1).to_dict()
    
    if results:
        video_id = results[0]['id']
        video_link = f"https://www.youtube.com/watch?v={video_id}"
        return video_link
    
    return None

###TEST###
video_name = 'ME!'
video_link = get_video_link_by_name(video_name)
if video_link:
    print(f"Video link: {video_link}")
else:
    print(f"No video found for '{video_name}'")

Video link: https://www.youtube.com/watch?v=FuXNumBwDOM


In [8]:
import yt_dlp

def download_audio_from_youtube(video_url, output_file):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_file,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'opus',
            'preferredquality': '192',
        }],
        'ffmpeg_location': r'C:\ffmpeg\bin\ffmpeg.exe'  # Replace with the actual path to ffmpeg
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

###TEST###
video_url = 'https://www.youtube.com/watch?v=FuXNumBwDOM'
output_file = 'audio'

download_audio_from_youtube(video_url, output_file)


[youtube] Extracting URL: https://www.youtube.com/watch?v=FuXNumBwDOM
[youtube] FuXNumBwDOM: Downloading webpage
[youtube] FuXNumBwDOM: Downloading ios player API JSON
[youtube] FuXNumBwDOM: Downloading android player API JSON
[youtube] FuXNumBwDOM: Downloading m3u8 information
[info] FuXNumBwDOM: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    3.56MiB in 00:00:01 at 3.09MiB/s   
[ExtractAudio] Destination: audio.opus
Deleting original file audio (pass -k to keep)


In [9]:
import librosa
def get_audio_duration(file_path):
    audio, sr = librosa.load(file_path)
    duration = librosa.get_duration(y=audio, sr=sr)
    return duration

###TEST###
file = 'audio.opus'
duration = get_audio_duration(file)
print(f"Audio duration: {duration} seconds")


Audio duration: 248.69351473922902 seconds


## Loading initial dataset

In [10]:
import pandas as pd

data = pd.read_csv('initial_dataset.csv')
data

,Unnamed: 0,Artist,Song,Popular
0,0,Yung Bleu,Life Worth Living,0
1,1,Yung Bleu,Everytime I Blink,0
2,2,Yung Bleu,Good,0
3,3,Yung Bleu,Come By At 12,0
4,4,Yung Bleu,"No, I'm Not Ok",0
...,...,...,...,...
1993,1993,The Weeknd,Take My Breath,1
1994,1994,The Weeknd,Heartless,1
1995,1995,YoungBoy Never Broke Again,Bandit,1
1996,1996,Mariah Carey,All I Want For Christmas Is You,1


## Defining functions to extract chorus from an audio and extract chorus soundtrack features

In [11]:
import pandas as pd
import librosa
import numpy as np 
import scipy
from tqdm import tqdm
from pychorus import find_and_output_chorus

# Function to download audio from YouTube using the artist and song name
def download_audio(artist, song):
    video_link = get_video_link_by_name(f'{artist} - {song}')
    download_audio_from_youtube(video_link, 'audio')

# Function to find the chorus start time in seconds
def create_chorus_file():
    if get_audio_duration('audio.opus')>480:
        raise ValueError("Video duration exceeds 8 minutes.")
        
        
    chorus_duration = 15  
    while chorus_duration > 5:
        chorus_start_sec = find_and_output_chorus("audio.opus", "output.mp3", chorus_duration)
        if chorus_start_sec is not None:
            return chorus_duration
        
        chorus_duration -= 1
        
    raise ValueError("No chorus detected")

# Functions to extract features from the chorus
functions = {
    'mfcc': lambda audio, sr: librosa.feature.mfcc(y=audio, sr=sr),
    'chroma_stft': lambda audio, sr: librosa.feature.chroma_stft(y=audio, sr=sr),
    'chroma_cqt': lambda audio, sr: librosa.feature.chroma_cqt(y=audio, sr=sr),
    'chroma_cens': lambda audio, sr: librosa.feature.chroma_cens(y=audio, sr=sr),
    'rms': lambda audio, sr: librosa.feature.rms(y=audio),
    'spectral_centroid': lambda audio, sr: librosa.feature.spectral_centroid(y=audio, sr=sr),
    'spectral_bandwidth': lambda audio, sr: librosa.feature.spectral_bandwidth(y=audio, sr=sr),
    'spectral_contrast': lambda audio, sr: librosa.feature.spectral_contrast(y=audio, sr=sr),
    'spectral_rolloff': lambda audio, sr: librosa.feature.spectral_rolloff(y=audio, sr=sr),
    'tonnetz': lambda audio, sr: librosa.feature.tonnetz(y=audio, sr=sr),
    'zero_crossing_rate': lambda audio, sr: librosa.feature.zero_crossing_rate(y=audio)
}



# Function to extract a feature statistics from the chorus 
def extract_soundtrack_feature_statistics(audio_feature):
    min_val = np.min(audio_feature, axis=1)
    mean_val = np.mean(audio_feature, axis=1)
    median_val = np.median(audio_feature, axis=1)
    max_val = np.max(audio_feature, axis=1)
    std_val = np.std(audio_feature, axis=1)
    skew_val = scipy.stats.skew(audio_feature, axis=1)
    kurtosis_val = scipy.stats.kurtosis(audio_feature, axis=1)
    song_track_features = np.concatenate((min_val, mean_val, median_val, max_val, std_val, skew_val, kurtosis_val), axis=0)
    return song_track_features



## Downloading audio for each dataset row, extracting the audio chorus soundtrack features, and appending features on the row

In [12]:

new_features_dict = {}

# Iterate over the rows of the data
for index, row in tqdm(data.iterrows(), total=len(data)):
    artist = row['Artist']
    song = row['Song']

    try:
        # Download the song
        download_audio(artist, song)
        print(artist+' - '+ song)

        # create chorus file: "output.mp3"
        chorus_duration = create_chorus_file()
        new_features_dict['chorus_duration'] = chorus_duration

      
        chorus_audio, sr = librosa.load('output.mp3', sr=None)
        for function_name,function in functions.items():
                
            # Extract a soundtrack feature
            audio_feature = function(chorus_audio, sr)
            audio_feature_statistics = extract_soundtrack_feature_statistics(audio_feature)

            # Update the dataframe with the chorus extracted features and the chorus duration  
            for i in range(len(audio_feature_statistics)):
                new_features_dict[function_name + '_'+str(i+1)] = audio_feature_statistics[i]
                    
            data.loc[index, new_features_dict.keys()]=new_features_dict.values()
            
        if index%30 == 0:
            data.to_csv('updated_dataset.csv', index=False)
                

    except Exception as e:
        print(f"Error processing {artist} - {song}: {str(e)}")
        continue


# Save the updated dataframe to a new file
data.to_csv('updated_dataset.csv', index=False)


  0%|                                                                                         | 0/1998 [00:00<?, ?it/s]

[youtube] Extracting URL: https://www.youtube.com/watch?v=CBxmbsD7B9w
[youtube] CBxmbsD7B9w: Downloading webpage
[youtube] CBxmbsD7B9w: Downloading ios player API JSON
[youtube] CBxmbsD7B9w: Downloading android player API JSON
[youtube] CBxmbsD7B9w: Downloading m3u8 information
[info] CBxmbsD7B9w: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    3.05MiB in 00:00:01 at 1.97MiB/s   
[ExtractAudio] Destination: audio.opus
Deleting original file audio (pass -k to keep)
Yung Bleu -  Life Worth Living
Best chorus found at 2 min 27.94 sec


  0%|                                                                               | 1/1998 [00:10<6:05:46, 10.99s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=o_Cyi4-DRIk
[youtube] o_Cyi4-DRIk: Downloading webpage
[youtube] o_Cyi4-DRIk: Downloading ios player API JSON
[youtube] o_Cyi4-DRIk: Downloading android player API JSON
[youtube] o_Cyi4-DRIk: Downloading m3u8 information
[info] o_Cyi4-DRIk: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    2.87MiB in 00:00:01 at 2.14MiB/s   
[ExtractAudio] Destination: audio.opus
Deleting original file audio (pass -k to keep)
Yung Bleu - Everytime I Blink
Best chorus found at 1 min 54.96 sec


  0%|                                                                               | 2/1998 [00:21<5:53:03, 10.61s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▉| 1996/1998 [7:33:53<00:22, 11.33s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=aAkMkVFwAoo
[youtube] aAkMkVFwAoo: Downloading webpage
[youtube] aAkMkVFwAoo: Downloading ios player API JSON
[youtube] aAkMkVFwAoo: Downloading android player API JSON
[youtube] aAkMkVFwAoo: Downloading m3u8 information
[info] aAkMkVFwAoo: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    3.73MiB in 00:00:01 at 3.08MiB/s   
[ExtractAudio] Destination: audio.opus
Deleting original file audio (pass -k to keep)
Mariah Carey - All I Want For Christmas Is You
Best chorus found at 3 min 3.08 sec


100%|███████████████████████████████████████████████████████████████████████████▉| 1997/1998 [7:34:05<00:11, 11.43s/it]

[youtube] Extracting URL: https://www.youtube.com/watch?v=CTYXd__nbvo
[youtube] CTYXd__nbvo: Downloading webpage
[youtube] CTYXd__nbvo: Downloading ios player API JSON
[youtube] CTYXd__nbvo: Downloading android player API JSON
[youtube] CTYXd__nbvo: Downloading m3u8 information
[info] CTYXd__nbvo: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    3.51MiB in 00:00:01 at 3.07MiB/s   
[ExtractAudio] Destination: audio.opus
Deleting original file audio (pass -k to keep)
Nelly - Lil Bit
Best chorus found at 3 min 15.04 sec


100%|████████████████████████████████████████████████████████████████████████████| 1998/1998 [7:34:16<00:00, 13.64s/it]


In [41]:
data = pd.read_csv('updated_dataset.csv')
data

,Unnamed: 0,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7
0,0,Yung Bleu,Life Worth Living,0,15.0,-168.736359,5.612945,-46.288033,-12.984114,-45.338833,...,-0.910918,0.271219,-0.150716,0.003906,0.054431,0.042969,0.317871,0.046725,2.677196,9.437756
1,1,Yung Bleu,Everytime I Blink,0,15.0,-337.154755,-76.327301,-71.523430,-35.614014,-66.932922,...,2.094208,2.561486,0.839045,0.006348,0.128698,0.102539,0.636230,0.107655,2.251092,5.922227
2,2,Yung Bleu,Good,0,15.0,-254.621109,24.572058,-34.989925,-5.104433,-53.706528,...,1.539465,-0.597756,-0.869980,0.008301,0.073600,0.066895,0.189453,0.037484,0.698273,-0.030552
3,3,Yung Bleu,Come By At 12,0,15.0,-333.629181,-17.214834,-61.820679,-19.658878,-34.620010,...,-0.795442,-0.392328,-0.642823,0.019043,0.090011,0.069336,0.613281,0.085119,2.992625,10.424276
4,4,Yung Bleu,"No, I'm Not Ok",0,15.0,-352.187286,7.102346,-47.977741,-18.019344,-66.934158,...,-0.554509,0.076702,-0.246584,0.003418,0.081523,0.075684,0.320312,0.052529,1.434354,3.122120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,1993,The Weeknd,Take My Breath,1,15.0,-300.524750,-26.474453,-58.435997,-17.161482,-73.338455,...,13.458372,14.778180,15.247367,0.011230,0.089688,0.079590,0.444336,0.055364,2.461916,9.736430
1994,1994,The Weeknd,Heartless,1,15.0,-150.605148,3.214969,-65.217575,-18.195301,-46.784615,...,4.041279,1.541470,0.383287,0.010254,0.087478,0.074707,0.430176,0.053133,1.855015,6.663876
1995,1995,YoungBoy Never Broke Again,Bandit,1,15.0,-266.642822,-12.099280,-49.327835,-22.512226,-64.726395,...,0.750007,0.217016,0.164434,0.002930,0.052104,0.035889,0.472656,0.056945,3.777320,19.321537
1996,1996,Mariah Carey,All I Want For Christmas Is You,1,15.0,-104.419067,46.025436,-48.758640,-17.115240,-46.777035,...,-0.506791,0.264294,-0.910116,0.062500,0.153688,0.145996,0.440918,0.046688,2.247846,9.093356


In [69]:
###TEST###
download_audio_from_youtube(get_video_link_by_name('Nelly - Lil Bit'),'audio')

[youtube] Extracting URL: https://www.youtube.com/watch?v=CTYXd__nbvo
[youtube] CTYXd__nbvo: Downloading webpage
[youtube] CTYXd__nbvo: Downloading ios player API JSON
[youtube] CTYXd__nbvo: Downloading android player API JSON
[youtube] CTYXd__nbvo: Downloading m3u8 information
[info] CTYXd__nbvo: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of    3.51MiB in 00:00:01 at 3.01MiB/s   
[ExtractAudio] Destination: audio.opus
Deleting original file audio (pass -k to keep)


In [70]:
###TEST###
chorus_start_sec = find_and_output_chorus("audio.opus", "output.mp3",  3)
chorus_start_sec

Best chorus found at 3 min 29.53 sec


209.5304602704152

In [71]:
###TEST###

import librosa
import numpy as np 
import scipy
# Load the chorus audio file (replace with your audio file)
chorus_audio, sr = librosa.load('output.mp3', sr=None)

# Extract the audio features from the chorus segment
# mfcc = librosa.feature.mfcc(y=chorus_audio, sr=sr)
# chroma_stft = librosa.feature.chroma_stft(y=chorus_audio, sr=sr)
# chroma_cqt = librosa.feature.chroma_cqt(y=chorus_audio, sr=sr)
# chroma_cens = librosa.feature.chroma_cens(y=chorus_audio, sr=sr)
# rms = librosa.feature.rms(y=chorus_audio)
# spectral_centroid = librosa.feature.spectral_centroid(y=chorus_audio, sr=sr)
# spectral_bandwidth = librosa.feature.spectral_bandwidth(y=chorus_audio, sr=sr)
spectral_contrast = librosa.feature.spectral_contrast(y=chorus_audio, sr=sr)
# spectral_rolloff = librosa.feature.spectral_rolloff(y=chorus_audio, sr=sr)
# tonnetz = librosa.feature.tonnetz(y=chorus_audio, sr=sr)
# zero_crossing_rate = librosa.feature.zero_crossing_rate(y=chorus_audio)

# Calculate the desired statistics for each feature
min_val = np.min(spectral_contrast, axis=1)
mean_val = np.mean(spectral_contrast, axis=1)
median_val = np.median(spectral_contrast, axis=1)
max_val = np.max(spectral_contrast, axis=1)
std_val = np.std(spectral_contrast, axis=1)
skew_val = scipy.stats.skew(spectral_contrast, axis=1)
kurtosis_val = scipy.stats.kurtosis(spectral_contrast, axis=1)

# Concatenate the calculated statistics into a single feature vector
song_track_features = np.concatenate((min_val, mean_val, median_val, max_val, std_val, skew_val, kurtosis_val), axis=0)

In [72]:
###TEST###
len(song_track_features)

49

## Finalizing

In [73]:
data

,Unnamed: 0,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7
0,0,Yung Bleu,Life Worth Living,0,15.0,-168.736359,5.612945,-46.288033,-12.984114,-45.338833,...,-0.910918,0.271219,-0.150716,0.003906,0.054431,0.042969,0.317871,0.046725,2.677196,9.437756
1,1,Yung Bleu,Everytime I Blink,0,15.0,-337.154755,-76.327301,-71.523430,-35.614014,-66.932922,...,2.094208,2.561486,0.839045,0.006348,0.128698,0.102539,0.636230,0.107655,2.251092,5.922227
2,2,Yung Bleu,Good,0,15.0,-254.621109,24.572058,-34.989925,-5.104433,-53.706528,...,1.539465,-0.597756,-0.869980,0.008301,0.073600,0.066895,0.189453,0.037484,0.698273,-0.030552
3,3,Yung Bleu,Come By At 12,0,15.0,-333.629181,-17.214834,-61.820679,-19.658878,-34.620010,...,-0.795442,-0.392328,-0.642823,0.019043,0.090011,0.069336,0.613281,0.085119,2.992625,10.424276
4,4,Yung Bleu,"No, I'm Not Ok",0,15.0,-352.187286,7.102346,-47.977741,-18.019344,-66.934158,...,-0.554509,0.076702,-0.246584,0.003418,0.081523,0.075684,0.320312,0.052529,1.434354,3.122120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,1993,The Weeknd,Take My Breath,1,15.0,-300.524750,-26.474453,-58.435997,-17.161482,-73.338455,...,13.458372,14.778180,15.247367,0.011230,0.089688,0.079590,0.444336,0.055364,2.461916,9.736430
1994,1994,The Weeknd,Heartless,1,15.0,-150.605148,3.214969,-65.217575,-18.195301,-46.784615,...,4.041279,1.541470,0.383287,0.010254,0.087478,0.074707,0.430176,0.053133,1.855015,6.663876
1995,1995,YoungBoy Never Broke Again,Bandit,1,15.0,-266.642822,-12.099280,-49.327835,-22.512226,-64.726395,...,0.750007,0.217016,0.164434,0.002930,0.052104,0.035889,0.472656,0.056945,3.777320,19.321537
1996,1996,Mariah Carey,All I Want For Christmas Is You,1,15.0,-104.419067,46.025436,-48.758640,-17.115240,-46.777035,...,-0.506791,0.264294,-0.910116,0.062500,0.153688,0.145996,0.440918,0.046688,2.247846,9.093356


In [74]:
data.isnull().sum()

Unnamed: 0                0
Artist                    0
Song                      1
Popular                   0
chorus_duration         230
                       ... 
zero_crossing_rate_3    230
zero_crossing_rate_4    230
zero_crossing_rate_5    230
zero_crossing_rate_6    230
zero_crossing_rate_7    230
Length: 523, dtype: int64

In [75]:
data[data['Song'].isnull()]

,Unnamed: 0,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7
1071,1071,Justin Bieber,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
data = data.drop(1071)

In [77]:
data.isnull().sum()

Unnamed: 0                0
Artist                    0
Song                      0
Popular                   0
chorus_duration         229
                       ... 
zero_crossing_rate_3    229
zero_crossing_rate_4    229
zero_crossing_rate_5    229
zero_crossing_rate_6    229
zero_crossing_rate_7    229
Length: 523, dtype: int64

In [78]:
data

,Unnamed: 0,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7
0,0,Yung Bleu,Life Worth Living,0,15.0,-168.736359,5.612945,-46.288033,-12.984114,-45.338833,...,-0.910918,0.271219,-0.150716,0.003906,0.054431,0.042969,0.317871,0.046725,2.677196,9.437756
1,1,Yung Bleu,Everytime I Blink,0,15.0,-337.154755,-76.327301,-71.523430,-35.614014,-66.932922,...,2.094208,2.561486,0.839045,0.006348,0.128698,0.102539,0.636230,0.107655,2.251092,5.922227
2,2,Yung Bleu,Good,0,15.0,-254.621109,24.572058,-34.989925,-5.104433,-53.706528,...,1.539465,-0.597756,-0.869980,0.008301,0.073600,0.066895,0.189453,0.037484,0.698273,-0.030552
3,3,Yung Bleu,Come By At 12,0,15.0,-333.629181,-17.214834,-61.820679,-19.658878,-34.620010,...,-0.795442,-0.392328,-0.642823,0.019043,0.090011,0.069336,0.613281,0.085119,2.992625,10.424276
4,4,Yung Bleu,"No, I'm Not Ok",0,15.0,-352.187286,7.102346,-47.977741,-18.019344,-66.934158,...,-0.554509,0.076702,-0.246584,0.003418,0.081523,0.075684,0.320312,0.052529,1.434354,3.122120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,1993,The Weeknd,Take My Breath,1,15.0,-300.524750,-26.474453,-58.435997,-17.161482,-73.338455,...,13.458372,14.778180,15.247367,0.011230,0.089688,0.079590,0.444336,0.055364,2.461916,9.736430
1994,1994,The Weeknd,Heartless,1,15.0,-150.605148,3.214969,-65.217575,-18.195301,-46.784615,...,4.041279,1.541470,0.383287,0.010254,0.087478,0.074707,0.430176,0.053133,1.855015,6.663876
1995,1995,YoungBoy Never Broke Again,Bandit,1,15.0,-266.642822,-12.099280,-49.327835,-22.512226,-64.726395,...,0.750007,0.217016,0.164434,0.002930,0.052104,0.035889,0.472656,0.056945,3.777320,19.321537
1996,1996,Mariah Carey,All I Want For Christmas Is You,1,15.0,-104.419067,46.025436,-48.758640,-17.115240,-46.777035,...,-0.506791,0.264294,-0.910116,0.062500,0.153688,0.145996,0.440918,0.046688,2.247846,9.093356


In [79]:
data = data.drop('Unnamed: 0',axis=1)

In [80]:
data

,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7
0,Yung Bleu,Life Worth Living,0,15.0,-168.736359,5.612945,-46.288033,-12.984114,-45.338833,-22.118843,...,-0.910918,0.271219,-0.150716,0.003906,0.054431,0.042969,0.317871,0.046725,2.677196,9.437756
1,Yung Bleu,Everytime I Blink,0,15.0,-337.154755,-76.327301,-71.523430,-35.614014,-66.932922,-61.465141,...,2.094208,2.561486,0.839045,0.006348,0.128698,0.102539,0.636230,0.107655,2.251092,5.922227
2,Yung Bleu,Good,0,15.0,-254.621109,24.572058,-34.989925,-5.104433,-53.706528,-13.358343,...,1.539465,-0.597756,-0.869980,0.008301,0.073600,0.066895,0.189453,0.037484,0.698273,-0.030552
3,Yung Bleu,Come By At 12,0,15.0,-333.629181,-17.214834,-61.820679,-19.658878,-34.620010,-47.485634,...,-0.795442,-0.392328,-0.642823,0.019043,0.090011,0.069336,0.613281,0.085119,2.992625,10.424276
4,Yung Bleu,"No, I'm Not Ok",0,15.0,-352.187286,7.102346,-47.977741,-18.019344,-66.934158,-22.188637,...,-0.554509,0.076702,-0.246584,0.003418,0.081523,0.075684,0.320312,0.052529,1.434354,3.122120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,The Weeknd,Take My Breath,1,15.0,-300.524750,-26.474453,-58.435997,-17.161482,-73.338455,-49.489502,...,13.458372,14.778180,15.247367,0.011230,0.089688,0.079590,0.444336,0.055364,2.461916,9.736430
1994,The Weeknd,Heartless,1,15.0,-150.605148,3.214969,-65.217575,-18.195301,-46.784615,-24.025791,...,4.041279,1.541470,0.383287,0.010254,0.087478,0.074707,0.430176,0.053133,1.855015,6.663876
1995,YoungBoy Never Broke Again,Bandit,1,15.0,-266.642822,-12.099280,-49.327835,-22.512226,-64.726395,-21.528536,...,0.750007,0.217016,0.164434,0.002930,0.052104,0.035889,0.472656,0.056945,3.777320,19.321537
1996,Mariah Carey,All I Want For Christmas Is You,1,15.0,-104.419067,46.025436,-48.758640,-17.115240,-46.777035,-15.846478,...,-0.506791,0.264294,-0.910116,0.062500,0.153688,0.145996,0.440918,0.046688,2.247846,9.093356


In [81]:
data[(data['Popular']!=0) & (data['Popular']!=1)]

,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7


In [82]:
data[(data['chorus_duration']<=5) & (data['chorus_duration']>15)]

,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7


#### Checking for songs with a shorter chorus duration, as some may not actually be songs

In [86]:
for row in data[data['chorus_duration']==6].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")



Luke Bryan - Episode 2": https://www.youtube.com/watch?v=XRheK52T-ak
DJ Snake - I'M BORED 🤣🔊: https://www.youtube.com/watch?v=Z4eKJiYBrVM
SZA - Wavy: https://www.youtube.com/watch?v=ZUJ235WszPA
Nicky Youre - Youre Vlogs: Nicky Youre x Benson Boone Pulse Toure Part 1: https://www.youtube.com/watch?v=u-ZHLUjF1Z4
Billie Eilish - Not My Responsibility: https://www.youtube.com/watch?v=ZlvfYmfefSI
Diane Guerrero - Albums & Singles: https://www.youtube.com/watch?v=bBeZSuHI4Qc
GAYLE - Creep: https://www.youtube.com/watch?v=GeGMuhYMw0w
THE SCOTTS - RANDY MOSS FOR CACTUS JACK 6'S: https://www.youtube.com/watch?v=qwaaTjezSa4
Olivia Rodrigo - SOUR Album Trailer: https://www.youtube.com/watch?v=AI53uIAy_BQ
Stephanie Beatriz - Dance Like No One's Watching: https://www.youtube.com/watch?v=UrFH772ytzM
Duncan Laurence - Q&A with Duncan Laurence: https://www.youtube.com/watch?v=UaLpl8hqRtI
Sam Tompkins - THEY SUNG MY SONG BACK TO ME: https://www.youtube.com/watch?v=VE8fGogrND4
Gwen Stefani - Tura Eyewea

In [87]:
for row in data[data['chorus_duration']==7].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")



Zach Bryan - This Road I Know: https://www.youtube.com/watch?v=GJ1adJzGOeA
Adele - Make You Feel My Love: https://www.youtube.com/watch?v=0put0_a--Ng
24kGoldn - Behind the Scenes: https://www.youtube.com/watch?v=umXrG5Pka1U
Glass Animals - GLASS ANIMALS » JOURNEY TO DREAMLAND IRL » 💜💜💜: https://www.youtube.com/watch?v=OZEllPJp744
SHAED - COLDPLAY PERFORMANCE ANNIVERSARY: https://www.youtube.com/watch?v=-OJvPJoxeSY
Marshmello - Movimenta : https://www.youtube.com/watch?v=_jWXyUVCePQ
Tyga - WELL DONE FEVER: https://www.youtube.com/watch?v=m7NDJIE4PV8
Justin Bieber - 2 Much: https://www.youtube.com/watch?v=xFJjczkU4So
Coi Leray - Artist on the Rise: Coi Leray: https://www.youtube.com/watch?v=U8OzqH1e-NA
Lee Brice - Country on the River 2016: https://www.youtube.com/watch?v=L1n7kEB4UEI
Sam Hunt - Alaska 2022: https://www.youtube.com/watch?v=gU_4gZeXzmY
Bradley Cooper - Unboxing video of cloudy day clothing ✨: https://www.youtube.com/watch?v=pdsFTL-r8fo
Lewis Capaldi - My birthday song is..

In [88]:
for row in data[data['chorus_duration']==8].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")

Parmalee - Matt's Jeep Renegade: https://www.youtube.com/watch?v=qLydqhvbKWo
H.E.R. - "Damage", "We Made It", "Are You Gonna Go My Way": https://www.youtube.com/watch?v=h-yPKOQa0IM
Nicky Youre - Boston & Philadelphia: https://www.youtube.com/watch?v=ydDqZTZOp6I
Surfaces - Album Release Show Recap: https://www.youtube.com/watch?v=nj4eiSRD-kM
Kid Cudi - Cudderisback: https://www.youtube.com/watch?v=fRk6K-H1Lxc
Tones And I - UK/EU 2020 Week 1: https://www.youtube.com/watch?v=bUdmWFv9utU
BRS Kash - Do We Have A Problem: https://www.youtube.com/watch?v=RiHuUgN_vvs
Halsey - HBO MAX TRAILER: https://www.youtube.com/watch?v=90eBBlt5-rc
Migos - The Gender Reveal Song: https://www.youtube.com/watch?v=S7rtN2b4ypc
Black Eyed Peas - MAMACITA Houseparty with Ozuna & J.Rey Soul: https://www.youtube.com/watch?v=Cb7xb2B59vI
Ty Dolla $ign - Famous Excuses: https://www.youtube.com/watch?v=G5EnR0K5r_A
Gunna - It's Just Us: https://www.youtube.com/watch?v=QCXRNxe4Jws
YoungBoy Never Broke Again - Dirty Thug

In [89]:
for row in data[data['chorus_duration']==9].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")

Bailey Zimmerman - God's Gonna Cut You Down: https://www.youtube.com/watch?v=91SimEgWj54
Bazzi - I’m Joining The NBA: https://www.youtube.com/watch?v=_F10p49vRPM
Nicky Youre - Detroit x New York x Poughkeepsie: https://www.youtube.com/watch?v=ueTuQ96sAUM
Gucci Mane - Gucci Flow: https://www.youtube.com/watch?v=PSspPUcCv-k
CJ - Marlboro Man 45033278946384765: https://www.youtube.com/watch?v=r229gH52z1g
Beyonce - Still: https://www.youtube.com/watch?v=NU5GGiFK5Bc
Jhene Aiko - FREE RIC: https://www.youtube.com/watch?v=EUvKqjngC-w
Jhene Aiko - Hard Times: https://www.youtube.com/watch?v=kLTwNhHVc7Q
NEIKED - ‘Best Of Hard Drive’ Album Announce: https://www.youtube.com/watch?v=UmBAj5wOP1o
Pop Smoke - Don’t Know Em: https://www.youtube.com/watch?v=w4oJdfFc4t0
Megan Thee Stallion - Thot Shit: https://www.youtube.com/watch?v=KynkMn5Hv3Q


In [6]:
for row in data[data['chorus_duration']==10].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")

StaySolidRocky - LoudStream: https://www.youtube.com/watch?v=9-k7vUlqmmw
THE ANXIETY: WILLOW - Are You Afraid: https://www.youtube.com/watch?v=5ywKaPlCIvo
Billie Eilish - my future: https://www.youtube.com/watch?v=Dm9Zf1WYQ_A
Carly Pearce - Sleigh Ride: https://www.youtube.com/watch?v=du9u7eCadCg
Chris Brown - Under The Influence Tour: Episode Two: https://www.youtube.com/watch?v=OLjzVV36FSQ
JNR CHOI - YOU SHOULD SHOOT THIS: https://www.youtube.com/watch?v=_eA4wGBUY9E
Imagine Dragons - Outro: https://www.youtube.com/watch?v=GZvA0bWtl5A
Migos - Ima Dogg: https://www.youtube.com/watch?v=v4XRhj734I0
Sam Tompkins - FADED WITH MY FANS: https://www.youtube.com/watch?v=51DI1skm-3Y
Sheck Wes - 2017 Freestyle: https://www.youtube.com/watch?v=gGxBT6IbStw
Sheck Wes - BEEN BALLIN: https://www.youtube.com/watch?v=DGoqaJtkMqE
Pooh Shiesty - Drop Some Shit: https://www.youtube.com/watch?v=qZiRpQdzK94
Pooh Shiesty - Big 13 Gang: https://www.youtube.com/watch?v=mS3rQrJaU2Y
Mustard - A Catastrophe We're

In [10]:
for row in data[data['chorus_duration']==11].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")

Carolina Gaitan - Popurrí de Selena: https://www.youtube.com/watch?v=jnbNEFaXV9o
Florida Georgia Line - Women: https://www.youtube.com/watch?v=EnnGRz90JnE
Glass Animals - Intruxx: https://www.youtube.com/watch?v=aTLvv54ik3s
Offset - "The Tonight Show Rehearsal": https://www.youtube.com/watch?v=NXcSfmgONFY
Lady Gaga - I've Got You Under My Skin: https://www.youtube.com/watch?v=xyTa_gJkYwI
Joji - Upgrade: https://www.youtube.com/watch?v=DoE_le4Te9U
Duncan Laurence - Sleeping On The Phone: https://www.youtube.com/watch?v=wUsrmZpJ4eM
Nicki Minaj - I Get Crazy: https://www.youtube.com/watch?v=9hPjBrkcRWo
Moneybagg Yo - GO!: https://www.youtube.com/watch?v=NYWDgu5sc6A


In [11]:
for row in data[data['chorus_duration']==12].iterrows():
    artist = row[1]['Artist']
    song = row[1]['Song']
    print(f"{artist} - {song}: {get_video_link_by_name(f'{artist} - {song}')}")

Calboy - HOUSE OF CARDS Dir. by Nick Rodriguez: https://www.youtube.com/watch?v=7d-6SGJLUx4
J Balvin - En Alta: https://www.youtube.com/watch?v=CMJO_zy745Y
Ritt Momney - Screwtape: https://www.youtube.com/watch?v=I98rdmZJBmQ
Florida Georgia Line - Hubbard Hills Episode 1: https://www.youtube.com/watch?v=oYZGxSa5Bto
Niko Moon - 2020 Recap: https://www.youtube.com/watch?v=KD_uIsAJjzU
SZA - Awkward: https://www.youtube.com/watch?v=p_11tLegwF0
Jason Aldean - Halloween 2020: https://www.youtube.com/watch?v=DJ8AlugQpEo
Ellie Goulding - Ode To Myself: https://www.youtube.com/watch?v=rFCa3H4Xndo
Billie Eilish - GOLDWING: https://www.youtube.com/watch?v=JpL7scu-EmY
Elton John - Rocketman – Crocodile Rock Scene: https://www.youtube.com/watch?v=Pdotum3y9cM
Tyler Cole - CULT CLASSIC: https://www.youtube.com/watch?v=fkPbcCW8w4k
Bad Bunny - De Museo: https://www.youtube.com/watch?v=ss0Un7fa-Mw
Bad Bunny - Un Coco: https://www.youtube.com/watch?v=lH3d69sbvX4
Cardi B - Press: https://www.youtube.com/w

## Saving Dataset

In [37]:
data.to_csv('final_dataset.csv', index=False)

In [38]:
pd.read_csv('final_dataset.csv')

,Artist,Song,Popular,chorus_duration,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,...,tonnetz_40,tonnetz_41,tonnetz_42,zero_crossing_rate_1,zero_crossing_rate_2,zero_crossing_rate_3,zero_crossing_rate_4,zero_crossing_rate_5,zero_crossing_rate_6,zero_crossing_rate_7
0,Yung Bleu,Life Worth Living,0,15.0,-168.736359,5.612945,-46.288033,-12.984114,-45.338833,-22.118843,...,-0.910918,0.271219,-0.150716,0.003906,0.054431,0.042969,0.317871,0.046725,2.677196,9.437756
1,Yung Bleu,Everytime I Blink,0,15.0,-337.154755,-76.327301,-71.523430,-35.614014,-66.932922,-61.465141,...,2.094208,2.561486,0.839045,0.006348,0.128698,0.102539,0.636230,0.107655,2.251092,5.922227
2,Yung Bleu,Good,0,15.0,-254.621109,24.572058,-34.989925,-5.104433,-53.706528,-13.358343,...,1.539465,-0.597756,-0.869980,0.008301,0.073600,0.066895,0.189453,0.037484,0.698273,-0.030552
3,Yung Bleu,Come By At 12,0,15.0,-333.629181,-17.214834,-61.820679,-19.658878,-34.620010,-47.485634,...,-0.795442,-0.392328,-0.642823,0.019043,0.090011,0.069336,0.613281,0.085119,2.992625,10.424276
4,Yung Bleu,"No, I'm Not Ok",0,15.0,-352.187286,7.102346,-47.977741,-18.019344,-66.934158,-22.188637,...,-0.554509,0.076702,-0.246584,0.003418,0.081523,0.075684,0.320312,0.052529,1.434354,3.122120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,The Weeknd,Take My Breath,1,15.0,-300.524750,-26.474453,-58.435997,-17.161482,-73.338455,-49.489502,...,13.458372,14.778180,15.247367,0.011230,0.089688,0.079590,0.444336,0.055364,2.461916,9.736430
1993,The Weeknd,Heartless,1,15.0,-150.605148,3.214969,-65.217575,-18.195301,-46.784615,-24.025791,...,4.041279,1.541470,0.383287,0.010254,0.087478,0.074707,0.430176,0.053133,1.855015,6.663876
1994,YoungBoy Never Broke Again,Bandit,1,15.0,-266.642822,-12.099280,-49.327835,-22.512226,-64.726395,-21.528536,...,0.750007,0.217016,0.164434,0.002930,0.052104,0.035889,0.472656,0.056945,3.777320,19.321537
1995,Mariah Carey,All I Want For Christmas Is You,1,15.0,-104.419067,46.025436,-48.758640,-17.115240,-46.777035,-15.846478,...,-0.506791,0.264294,-0.910116,0.062500,0.153688,0.145996,0.440918,0.046688,2.247846,9.093356
